- # **Atividade:**

1. Crie uma consulta que mostre nesta ordem, Nome, Estados e Status do cliente.
2. Crie uma consulta que mostre apenas os clientes do Status "platinum" e "gold".
3. Demonstre quanto cada Status de Clientes representa em vendas?

- Resolver com DataFrames.
- Utilizar joins se necessário.


![](./images/der-atividade.png)

In [24]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

from pyspark.sql.types import *

- ## Setting data path to work with Data
    - ### In case **you haven't installed** dotenv library:

In [5]:
!pip3 install python-dotenv

- ### **After installing** dotenv library:

In [25]:
from dotenv import dotenv_values


env_vars = dotenv_values('C:\\Users\\017924631\Desktop\\working-tree\\learning-courses\\spark\\ambvir\\.env')

def return_tablePath(data_arc: str):
    # could tranform it into a list and access by index:  - list(env_vars.values())[0]
    data_root_path = env_vars['DATA_ROOT_PATH'] + '\\'
    data_path = data_root_path + data_arc

    return data_path

- ## Importing Data

In [26]:
parq_clients = spark.read.format("parquet").load(return_tablePath('Clientes.parquet'))

parq_clients.show(5)

+---------+--------------------+------+------+--------+
|ClienteID|             Cliente|Estado|Genero|  Status|
+---------+--------------------+------+------+--------+
|        1|Adelina Buenaventura|    RJ|     M|  Silver|
|        2|        Adelino Gago|    RJ|     M|  Silver|
|        3|     Adolfo Patrício|    PE|     M|  Silver|
|        4|    Adriana Guedelha|    RO|     F|Platinum|
|        5|       Adélio Lisboa|    SE|     M|  Silver|
+---------+--------------------+------+------+--------+
only showing top 5 rows



- ## Working with Data

- ### Resolving question 1:
    - Crie uma consulta que mostre nesta ordem, Nome, Estados e Status do cliente:

In [29]:
parq_clients.select("Cliente", "Estado", "Status").show(5)

+--------------------+------+--------+
|             Cliente|Estado|  Status|
+--------------------+------+--------+
|Adelina Buenaventura|    RJ|  Silver|
|        Adelino Gago|    RJ|  Silver|
|     Adolfo Patrício|    PE|  Silver|
|    Adriana Guedelha|    RO|Platinum|
|       Adélio Lisboa|    SE|  Silver|
+--------------------+------+--------+
only showing top 5 rows



- ### Resolving question 2:
    - Crie uma consulta que mostre apenas os clientes do Status "platinum" e "gold".

In [42]:
from pyspark.sql import functions as Func

parq_clients.select("Cliente", "Status").where((Func.col("Status") == "Platinum") | (Func.col("Status") == "Gold")).show()

+-------------------+--------+
|            Cliente|  Status|
+-------------------+--------+
|   Adriana Guedelha|Platinum|
|   Alberto Monsanto|    Gold|
|      Anna Carvajal|    Gold|
|      Bento Quintão|    Gold|
|      Carminda Dias|    Gold|
|      Cláudio Jorge|    Gold|
|    Dionísio Saltão|    Gold|
|   Firmino Meireles|    Gold|
|      Flor Vilanova|Platinum|
|Honorina Villaverde|    Gold|
|    Ibijara Botelho|Platinum|
|  Iracema Rodríguez|    Gold|
|         Joana Ataí|Platinum|
+-------------------+--------+



- ### Resolving question 3:
    - Demonstre quanto cada Status de Clientes representa em vendas?

In [43]:
parq_vendas = spark.read.format("parquet").load(return_tablePath('Vendas.parquet'))
parq_vendas.show(5)

+--------+----------+---------+--------+--------+
|VendasID|VendedorID|ClienteID|    Data|   Total|
+--------+----------+---------+--------+--------+
|       1|         1|       91|1/1/2019|  8053.6|
|       2|         6|      185|1/1/2020|   150.4|
|       3|         7|       31|2/1/2020|  6087.0|
|       4|         5|       31|2/1/2019| 13828.6|
|       5|         5|       31|3/1/2018|26096.66|
+--------+----------+---------+--------+--------+
only showing top 5 rows



In [51]:
parq_vendas_clientes = parq_clients.join(parq_vendas, parq_clients.ClienteID == parq_vendas.ClienteID, 'inner')
parq_vendas_clientes.show(5)

+---------+-----------------+------+------+------+--------+----------+---------+--------+--------+
|ClienteID|          Cliente|Estado|Genero|Status|VendasID|VendedorID|ClienteID|    Data|   Total|
+---------+-----------------+------+------+------+--------+----------+---------+--------+--------+
|       91|   Cosme Zambujal|    BA|     M|Silver|       1|         1|       91|1/1/2019|  8053.6|
|      185|Gertrudes Hidalgo|    PA|     F|Silver|       2|         6|      185|1/1/2020|   150.4|
|       31| Antão Corte-Real|    ES|     M|Silver|       3|         7|       31|2/1/2020|  6087.0|
|       31| Antão Corte-Real|    ES|     M|Silver|       4|         5|       31|2/1/2019| 13828.6|
|       31| Antão Corte-Real|    ES|     M|Silver|       5|         5|       31|3/1/2018|26096.66|
+---------+-----------------+------+------+------+--------+----------+---------+--------+--------+
only showing top 5 rows



In [70]:
parq_vendas_clientes_orderBy = parq_vendas_clientes.select('Cliente', 'Total').groupBy('Cliente').agg(Func.sum('Total')).orderBy(Func.col('sum(Total)').desc())

parq_vendas_clientes_orderBy.select('Cliente', Func.round('sum(Total)', 2)).show(5)

+-----------------+--------------------+
|          Cliente|round(sum(Total), 2)|
+-----------------+--------------------+
| Antão Corte-Real|           640268.07|
|   Brígida Gusmão|           366502.71|
|    Cidália Miera|           271000.02|
|Gertrudes Hidalgo|             96443.3|
|   Ilduara Chávez|            80674.51|
+-----------------+--------------------+
only showing top 5 rows

